In [6]:
import cv2
import numpy as np 
import pandas as pd 
import os
from PIL import Image
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
import tensorflow.keras as K
from keras.applications import MobileNetV2
from tensorflow.keras import datasets, layers, models, losses, Model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pre
from glob import glob
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
os.environ["CUDA_VISIBLE_DEVICES"] = '0'


In [2]:
train_df=pd.read_csv('../../data/classificationDDH/train_aug_classification_dataset.csv')
Train_img_list=train_df['file name'].to_list()
Train_label_list=train_df['standard class'].to_list()
Train_case_list=train_df['case'].to_list()


In [5]:
test_df=pd.read_csv('../../data/classificationDDH/test_aug_classification_dataset.csv')
Test_img_list=train_df['file name'].to_list()
Test_label_list=train_df['standard class'].to_list()
Test_case_list=train_df['case'].to_list()

,case,file name,standard class
0,2,00683_0.png,1
1,2,00683_1.png,1
2,2,00683_2.png,1
3,2,00683_3.png,1
4,2,00683_4.png,1
...,...,...,...
220165,19,00345_1.png,0
220166,19,00345_2.png,0
220167,19,00345_3.png,0
220168,19,00345_4.png,0


In [54]:
size=224
x_train = np.zeros((len(Train_img_list),size,size,3))
for i in range(len(Train_img_list)):
    x_train[i] =np.array(Image.open(Train_img_path+Train_img_list[i]).resize((size,size)))
x_train=x_train/255
y_train=np.array(Train_label_list)

x_test = np.zeros((len(Test_img_list),size,size,3))
for i in range(len(Test_img_list)):
    x_test[i] =np.array(Image.open(Test_img_path+Test_img_list[i]).resize((size,size)))
x_test=x_test/255
y_test=np.array(Test_label_list)

x_val = np.zeros((len(Val_img_list),size,size,3))
for i in range(len(Val_img_list)):
    x_val[i] =np.array(Image.open(Val_img_path+Val_img_list[i]).resize((size,size)))
x_val=x_val/255
y_val=np.array(Val_label_list)

In [58]:
def batch_generator(image, label, batchsize):
    N = len(image)
    indices = np.arange(N)  # 0부터 N-1까지의 인덱스 배열 생성
    np.random.shuffle(indices)  # 인덱스 배열을 무작위로 섞음

    i = 0
    while True:
        if i + batchsize <= N:
            batch_indices = indices[i:i+batchsize]
            i = i + batchsize
        else:  # 남은 데이터가 batchsize보다 작을 때, 배열을 wrap around하여 다시 섞음
            batch_indices = np.concatenate((indices[i:], indices[:batchsize - (N - i)]))
            i = batchsize - (N - i)

            np.random.shuffle(indices)  # 다음 에포크를 위해 인덱스 배열을 무작위로 섞음

        yield image[batch_indices], label[batch_indices]
        
checkpoint_filepath = "../../model/skip/MobileNetV2_checkpoints.h5"
checkpoint_filepath1 = "../../model/skip/MobileNetV2_checkpoints.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only= True
)
model_checkpoint_callback1 = tf.keras.callbacks.ModelCheckpoint(
    monitor='val_accuracy',
    filepath=checkpoint_filepath1,
    save_best_only= True
)
class_weight_ratio=compute_class_weight(class_weight = "balanced" , 
                     classes=np.unique(y_train), 
                     y = y_train)
class_weight = {0:class_weight_ratio[0],1:class_weight_ratio[1]}

input_t=K.Input(shape=(size,size, 3))
input_tensor = layers.experimental.preprocessing.Resizing(size, size, interpolation="bilinear", input_shape=x_train.shape[1:])(input_t)
ResNet=MobileNetV2(include_top=True,weights='imagenet',input_tensor=input_tensor)
model = K.models.Sequential()
model.add(ResNet)
model.add(tf.keras.layers.Dropout(.2, input_shape=(64,)))
model.add(K.layers.Dense(64, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(K.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=K.optimizers.Adam(lr=2e-3),
                loss=tf.keras.losses.binary_crossentropy,
                metrics=["accuracy"])
histo=model.fit(
    batch_generator(x_train,y_train,64),
    validation_data=(x_val,y_val),
    epochs=500,
    steps_per_epoch=len(x_train)//64,
    callbacks=[model_checkpoint_callback,model_checkpoint_callback1],
    shuffle=True,
    class_weight=class_weight
)
model.save('../../model/skip/MobileNetV2_GAN.h5')

Epoch 1/500


2023-08-07 17:02:26.519532: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


674/674 [==============================] - 71s 79ms/step - loss: 0.2391 - accuracy: 0.9265 - val_loss: 0.3271 - val_accuracy: 0.9335
Epoch 2/500
674/674 [==============================] - 47s 69ms/step - loss: 0.0944 - accuracy: 0.9629 - val_loss: 5.4547 - val_accuracy: 0.2247
Epoch 3/500
674/674 [==============================] - 48s 71ms/step - loss: 0.0711 - accuracy: 0.9721 - val_loss: 13.6639 - val_accuracy: 0.0665
Epoch 4/500
674/674 [==============================] - 49s 72ms/step - loss: 0.0562 - accuracy: 0.9784 - val_loss: 3.4693 - val_accuracy: 0.5670
Epoch 5/500
674/674 [==============================] - 47s 69ms/step - loss: 0.0518 - accuracy: 0.9808 - val_loss: 6.8095 - val_accuracy: 0.0665
Epoch 6/500
674/674 [==============================] - 49s 73ms/step - loss: 0.0407 - accuracy: 0.9845 - val_loss: 1.0138 - val_accuracy: 0.8469
Epoch 7/500
674/674 [==============================] - 49s 72ms/step - loss: 0.0367 - accuracy: 0.9862 - val_loss: 1.4479 - val_accuracy: 0.7